# Create train, validate, and test data sets
```
Andrew E. Davidson
aedavids@ucsc.edu
```

ref: [sklearn.model_selection.StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html#sklearn.model_selection.StratifiedShuffleSplit)

## AEDWIP
- focus on groupByGenes
- later create NumReads data sets
- TCGA col data does not have tissue id. 
    * tmp work around, copy Cohort colemn as tissue_id
    * maybe there is additional meta data on TCGA sigh
    * follow up with Toil compondum. what did they do?
```
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/matrices/NumReads/
gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/matrices/groupByGeneId/
```

In [1]:
import numpy as np
import pandas as pd
aedwip does not work from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
UBER_TCGA_groupByGeneMatrixURL = "$WORKSPACE_BUCKET/"

TCGA_colDataDirURL       = "$WORKSPACE_BUCKET/data/colData"
debug_TCGA_colDataDirURL = "$WORKSPACE_BUCKET/data/colDataDebug"

UBER_TCGA_colDataMatrixURL      = TCGA_colDataDirURL + "/uberColData.csv"
debug_UBER_TCGA_colDataMatrixURL = debug_TCGA_colDataDirURL + "/uberColDataDebug.csv"

localColDataTmp =  "data/colData/"

In [3]:
%%time
# create test and debug data

def createTestData():
    debugColDataList = [
        "$TCGA_colDataDirURL/TCGA_ACC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab_colData.csv",
        "$TCGA_colDataDirURL/TCGA_BLCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab_colData.csv"
    ]
    
    ! rm -rf debug_TCGA_colDataDirURL
    
    ! gsutil -m cp \
        $TCGA_colDataDirURL/TCGA_ACC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab_colData.csv \
        $TCGA_colDataDirURL/TCGA_BLCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab_colData.csv \
        $debug_TCGA_colDataDirURL
    
createTestData() 

Copying gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colData/TCGA_ACC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab_colData.csv [Content-Type=text/csv]...
Copying gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colData/TCGA_BLCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab_colData.csv [Content-Type=text/csv]...
/ [2/2 files][ 30.4 KiB/ 30.4 KiB] 100% Done                                    
Operation completed over 2 objects/30.4 KiB.                                     
CPU times: user 97.9 ms, sys: 26.3 ms, total: 124 ms
Wall time: 5.29 s


## Combine all the TCGA data set

In [4]:
%%time
def loadCSV(matrixURL, dataDirURL, localTmpDir):
    '''
    localTmpDir
        a path to a directory loadCSV is free to create, or delete
        
    returns pandas data frame
    
    if matrix URL does not exits 
        create a pandas data frame from file under dataDirURL 
        saves the dataframe to matrixURL
    '''
    
    exitCodeList = ! (gsutil -q stat $matrixURL; echo $?)
    exitCode = int(exitCodeList[0])
    retDF = None
    if exitCode == 0:
        # matrixURL exits
        print("reading:{}".format(matrixURL))
        ! 'rm' -rf $localTmpDir
        ! mkdir -p $localTmpDir
        ! gsutil -m cp "$matrixURL" $localTmpDir 
        file = !ls $localTmpDir
        p = localTmpDir + "/" + file[0]
        retDF = pd.read_csv(p)
       
    else:
        print("create matrix: {}".format(matrixURL))
        ! 'rm' -rf $localTmpDir
        ! mkdir -p $localTmpDir
        ! gsutil -m cp "$dataDirURL/*" $localTmpDir     
        fileList = ! ls $localTmpDir
        
        # read files and "union" them
        retDF = pd.DataFrame()
        for f in fileList:
            p = localTmpDir + "/" + f
            print(p)
            print("\n**** aedwip remove head!!!!")
            tmpDF = pd.read_csv(p).head(20)
            retDF = pd.concat([retDF, tmpDF])
            
        print("!!!! do not hard code csv")
        p = localTmpDir + "/" "matrix.csv"
        retDF.to_csv(p, index=False)
        
        # read with out index column
        retDF = pd.read_csv(p)

        
        ! gsutil -m cp $p $matrixURL
        
    # clean up
    ! 'rm' -rf $localTmpDir
            
    return retDF
            

colDataDF = loadCSV(debug_UBER_TCGA_colDataMatrixURL, debug_TCGA_colDataDirURL, localColDataTmp)
colDataDF

reading:$WORKSPACE_BUCKET/data/colDataDebug/uberColDataDebug.csv
Copying gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colDataDebug/uberColDataDebug.csv...
/ [1/1 files][  2.4 KiB/  2.4 KiB] 100% Done                                    
Operation completed over 1 objects/2.4 KiB.                                      
CPU times: user 110 ms, sys: 63.3 ms, total: 173 ms
Wall time: 9.81 s


,entity:sample_id,entity:participant_id,tcga_sample_id,Cohort,Age,Gender,sample_type
0,ACC-OR-A5J1-TP,ACC-OR-A5J1,TCGA-OR-A5J1-01,ACC,58,male,TP
1,ACC-OR-A5J2-TP,ACC-OR-A5J2,TCGA-OR-A5J2-01,ACC,44,female,TP
2,ACC-OR-A5J3-TP,ACC-OR-A5J3,TCGA-OR-A5J3-01,ACC,23,female,TP
3,ACC-OR-A5J5-TP,ACC-OR-A5J5,TCGA-OR-A5J5-01,ACC,30,male,TP
4,ACC-OR-A5J6-TP,ACC-OR-A5J6,TCGA-OR-A5J6-01,ACC,29,female,TP
5,ACC-OR-A5J7-TP,ACC-OR-A5J7,TCGA-OR-A5J7-01,ACC,30,female,TP
6,ACC-OR-A5J8-TP,ACC-OR-A5J8,TCGA-OR-A5J8-01,ACC,66,male,TP
7,ACC-OR-A5J9-TP,ACC-OR-A5J9,TCGA-OR-A5J9-01,ACC,22,female,TP
8,ACC-OR-A5JA-TP,ACC-OR-A5JA,TCGA-OR-A5JA-01,ACC,53,female,TP
9,ACC-OR-A5JB-TP,ACC-OR-A5JB,TCGA-OR-A5JB-01,ACC,52,male,TP


In [5]:
! gsutil ls -l $debug_TCGA_colDataDirURL

      4762  2022-06-09T01:55:35Z  gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colDataDebug/TCGA_ACC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab_colData.csv
     26362  2022-06-09T01:55:35Z  gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colDataDebug/TCGA_BLCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab_colData.csv
      2490  2022-06-09T01:38:49Z  gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/colDataDebug/uberColDataDebug.csv
TOTAL: 3 objects, 33614 bytes (32.83 KiB)


## Split data into train, validate, and test data sets
### convert  'Cohort', 'Gender' and 'sample_type' to  factors

In [6]:
colDataDF["Cohort"]      = colDataDF["Cohort"].astype('category')
colDataDF["Gender"]      = colDataDF["Gender"].astype('category')
colDataDF["sample_type"] = colDataDF["sample_type"].astype('category')
colDataDF.dtypes

entity:sample_id           object
entity:participant_id      object
tcga_sample_id             object
Cohort                   category
Age                         int64
Gender                   category
sample_type              category
dtype: object

### Create a label we can use to balance data sets by  Cohort, Gender, and sample_type
<span style="color:red">These columns do not match GTExColData</span> Our tmp work around for 1vsAll is to copy cohort column as tissue_id. Maybe we can find additional data on the TCGA websie
```
aedavids@mustard $ head /private/groups/kimlab/GTEx/GTExTrainColData.csv 
sample_id,participant_id,tissue_id,tissue_site_detail,sex,age
GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F,Adipose_Subcutaneous,Adipose - Subcutaneous,Female,66
GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F,Artery_Tibial,Artery - Tibial,Female,66
GTEX-1117F-0726-SM-5GIEN,GTEX-1117F,Heart_Atrial_Appendage,Heart - Atrial Appendage,Female,66
GTEX-1117F-2826-SM-5GZXL,GTEX-1117F,Breast_Mammary_Tissue,Breast - Mammary Tissue,Female,66
GTEX-1117F-3226-SM-5N9CT,GTEX-1117F,Brain_Cortex,Brain - Cortex,Female,66
GTEX-111CU-0326-SM-5GZXO,GTEX-111CU,Lung,Lung,Male,57
GTEX-111CU-0426-SM-5GZY1,GTEX-111CU,Spleen,Spleen,Male,57
GTEX-111CU-0526-SM-5EGHK,GTEX-111CU,Pancreas,Pancreas,Male,57
GTEX-111CU-0626-SM-5EGHL,GTEX-111CU,Esophagus_Muscularis,Esophagus - Muscularis,Male,57
```

```
$ head -n 1 TCGA_ACC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab/sample.tsv | sed -e 's/\t/\n/g'
entity:sample_id
participant
sample_type
snp__genome_wide_snp_6__broad_mit_edu__Level_1__image_acquisition__data
WXS_bai_path
clin__bio__nationwidechildrens_org__Level_1__biospecimen__clin
WXS_bam_analysis_id
VALIDATION_bai_path
snp__genome_wide_snp_6__broad_mit_edu__Level_2__birdseed_genotype__birdseed
snp__genome_wide_snp_6__broad_mit_edu__Level_3__segmented_scna_hg18__seg
clin__bio__nationwidechildrens_org__Level_1__clinical__clin
snp__genome_wide_snp_6__broad_mit_edu__Level_3__segmented_scna_hg19__seg
snp__genome_wide_snp_6__broad_mit_edu__Level_3__segmented_scna_minus_germline_cnv_hg18__seg
VALIDATION_bam_path
VALIDATION_bam_analysis_id
WXS_bam_path
snp__genome_wide_snp_6__broad_mit_edu__Level_3__segmented_scna_minus_germline_cnv_hg19__seg
tcga_sample_id
quantFile
mirnaseq__illuminahiseq_mirnaseq__bcgsc_ca__Level_3__miR_isoform_expression__data
mRNASeq_bai_path
tracerel__illuminaga_dnaseq__bcm_edu__Level_2__Mutation_Calling__maf
rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes__data
rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data
mRNASeq_bam_analysis_id
mRNASeq_fastq_analysis_id
mirnaseq__illuminahiseq_mirnaseq__bcgsc_ca__Level_3__miR_gene_expression__data
aux_info
mRNASeq_fastq_path
quantFilePaired
rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__exon_quantification__data
rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__junction_quantification__data
rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_isoforms_normalized__data
mRNASeq_bam_path
quantFilePaired3
methylation__humanmethylation450__jhu_usc_edu__Level_3__within_bioassay_data_set_function__data
rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_isoforms__data
aux_infoPaired3
aux_infoPaired
protein_exp__mda_rppa_core__mdanderson_org__Level_3__protein_normalization__data
```

In [7]:
a = colDataDF["Cohort"].astype(str).values
b = colDataDF["Gender"].astype(str).values
c = colDataDF["sample_type"].astype(str).values

colDataDF["label"] = ( a + "_" + b + "_" + c)
colDataDF["label"] = colDataDF["label"].astype('category')

# print("colDataDF['label'].categories : {}\n".format(colDataDF["label"].categories ))
print(colDataDF['label'].dtypes)
print(colDataDF['label'].unique())

colDataDF

category
['ACC_male_TP', 'ACC_female_TP', 'BLCA_male_TP', 'BLCA_female_TP']
Categories (4, object): ['ACC_female_TP', 'ACC_male_TP', 'BLCA_female_TP', 'BLCA_male_TP']


,entity:sample_id,entity:participant_id,tcga_sample_id,Cohort,Age,Gender,sample_type,label
0,ACC-OR-A5J1-TP,ACC-OR-A5J1,TCGA-OR-A5J1-01,ACC,58,male,TP,ACC_male_TP
1,ACC-OR-A5J2-TP,ACC-OR-A5J2,TCGA-OR-A5J2-01,ACC,44,female,TP,ACC_female_TP
2,ACC-OR-A5J3-TP,ACC-OR-A5J3,TCGA-OR-A5J3-01,ACC,23,female,TP,ACC_female_TP
3,ACC-OR-A5J5-TP,ACC-OR-A5J5,TCGA-OR-A5J5-01,ACC,30,male,TP,ACC_male_TP
4,ACC-OR-A5J6-TP,ACC-OR-A5J6,TCGA-OR-A5J6-01,ACC,29,female,TP,ACC_female_TP
5,ACC-OR-A5J7-TP,ACC-OR-A5J7,TCGA-OR-A5J7-01,ACC,30,female,TP,ACC_female_TP
6,ACC-OR-A5J8-TP,ACC-OR-A5J8,TCGA-OR-A5J8-01,ACC,66,male,TP,ACC_male_TP
7,ACC-OR-A5J9-TP,ACC-OR-A5J9,TCGA-OR-A5J9-01,ACC,22,female,TP,ACC_female_TP
8,ACC-OR-A5JA-TP,ACC-OR-A5JA,TCGA-OR-A5JA-01,ACC,53,female,TP,ACC_female_TP
9,ACC-OR-A5JB-TP,ACC-OR-A5JB,TCGA-OR-A5JB-01,ACC,52,male,TP,ACC_male_TP


In [8]:
colDataDF.groupby( ["label"] )['label'].agg('count')

label
ACC_female_TP     11
ACC_male_TP        9
BLCA_female_TP     2
BLCA_male_TP      18
Name: label, dtype: int64

### remove classremove classes that only have a few examples or are not balance

In [9]:
key = 'display.max_rows'
default = pd.get_option(key)
pd.set_option(key, None) # None
labelCountsSeries = colDataDF.groupby( ["label"] )['label'].agg('count')

In [10]:
print(labelCountsSeries.describe())
print()
bq = 0.05
bottomQuantile = labelCountsSeries.quantile(q=bq)
print("bottomQuantile {}% = {}".format(int(100 * bq), bottomQuantile))

count     4.000000
mean     10.000000
std       6.582806
min       2.000000
25%       7.250000
50%      10.000000
75%      12.750000
max      18.000000
Name: label, dtype: float64

bottomQuantile 5% = 3.0500000000000003


In [11]:
selectRowsBellow = labelCountsSeries < bottomQuantile
print("\nbellow bottomQuantile {}% = {}".format(int(100 * bq), bottomQuantile))
print(labelCountsSeries[ selectRowsBellow ])


bellow bottomQuantile 5% = 3.0500000000000003
label
BLCA_female_TP    2
Name: label, dtype: int64


In [12]:
selectRowsAbove =  [not x for x in selectRowsBellow]
labelsAboveThreshold = labelCountsSeries[ selectRowsAbove ]
labelsAboveThreshold.sort_values()

label
ACC_male_TP       9
ACC_female_TP    11
BLCA_male_TP     18
Name: label, dtype: int64

### split out a 60 % of the samples into a training set

In [13]:
theMeaningOfLife = 42
# n_splits = 1, we are not doing cross validation
split = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=theMeaningOfLife)
selectRows = colDataDF["label"].isin( labelsAboveThreshold.index )
X = colDataDF.loc[ selectRows  ].to_numpy()
y = colDataDF.loc[ selectRows, ['label'] ].to_numpy()
print("colDataDF.shape:{}".format(colDataDF.shape))
print("X.shape:{}".format(X.shape))
print("y.shape:{}".format(y.shape))

colDataDF.shape:(40, 8)
X.shape:(38, 8)
y.shape:(38, 1)


In [14]:
for train_index, not_train_index in split.split(X, y):
    X_train = X[train_index]
    X_not_train = X[not_train_index]
    
    y_train = y[train_index]
    y_not_train =y[not_train_index]
    
print("AEDWIP X_train.shape:{}".format(X_train.shape))
print("AEDWIP y_train.shape:{}".format(y_train.shape))
print()
print("AEDWIP X_not_train.shape:{}".format(X_not_train.shape))
print("AEDWIP y_not_train.shape:{}".format(y_not_train.shape))

print("")
print( y_train[0:5] )


trainColDataDF = pd.DataFrame( X_train, columns=colDataDF.columns)
trainColDataDF#.head()

AEDWIP X_train.shape:(22, 8)
AEDWIP y_train.shape:(22, 1)

AEDWIP X_not_train.shape:(16, 8)
AEDWIP y_not_train.shape:(16, 1)

[['ACC_male_TP']
 ['ACC_male_TP']
 ['ACC_female_TP']
 ['ACC_female_TP']
 ['ACC_female_TP']]


,entity:sample_id,entity:participant_id,tcga_sample_id,Cohort,Age,Gender,sample_type,label
0,ACC-OR-A5JK-TP,ACC-OR-A5JK,TCGA-OR-A5JK-01,ACC,49,male,TP,ACC_male_TP
1,ACC-OR-A5JG-TP,ACC-OR-A5JG,TCGA-OR-A5JG-01,ACC,61,male,TP,ACC_male_TP
2,ACC-OR-A5JA-TP,ACC-OR-A5JA,TCGA-OR-A5JA-01,ACC,53,female,TP,ACC_female_TP
3,ACC-OR-A5JL-TP,ACC-OR-A5JL,TCGA-OR-A5JL-01,ACC,36,female,TP,ACC_female_TP
4,ACC-OR-A5J3-TP,ACC-OR-A5J3,TCGA-OR-A5J3-01,ACC,23,female,TP,ACC_female_TP
5,BLCA-2F-A9KT-TP,BLCA-2F-A9KT,TCGA-2F-A9KT-01,BLCA,83,male,TP,BLCA_male_TP
6,BLCA-4Z-AA7Q-TP,BLCA-4Z-AA7Q,TCGA-4Z-AA7Q-01,BLCA,79,male,TP,BLCA_male_TP
7,BLCA-4Z-AA81-TP,BLCA-4Z-AA81,TCGA-4Z-AA81-01,BLCA,80,male,TP,BLCA_male_TP
8,ACC-OR-A5J6-TP,ACC-OR-A5J6,TCGA-OR-A5J6-01,ACC,29,female,TP,ACC_female_TP
9,BLCA-4Z-AA7W-TP,BLCA-4Z-AA7W,TCGA-4Z-AA7W-01,BLCA,55,male,TP,BLCA_male_TP


In [15]:
X_not_trainDF = pd.DataFrame(X_not_train, columns=colDataDF.columns)
X_not_trainDF

,entity:sample_id,entity:participant_id,tcga_sample_id,Cohort,Age,Gender,sample_type,label
0,BLCA-2F-A9KP-TP,BLCA-2F-A9KP,TCGA-2F-A9KP-01,BLCA,66,male,TP,BLCA_male_TP
1,BLCA-4Z-AA7S-TP,BLCA-4Z-AA7S,TCGA-4Z-AA7S-01,BLCA,66,male,TP,BLCA_male_TP
2,ACC-OR-A5JD-TP,ACC-OR-A5JD,TCGA-OR-A5JD-01,ACC,57,female,TP,ACC_female_TP
3,ACC-OR-A5JJ-TP,ACC-OR-A5JJ,TCGA-OR-A5JJ-01,ACC,65,male,TP,ACC_male_TP
4,BLCA-4Z-AA83-TP,BLCA-4Z-AA83,TCGA-4Z-AA83-01,BLCA,52,male,TP,BLCA_male_TP
5,ACC-OR-A5JB-TP,ACC-OR-A5JB,TCGA-OR-A5JB-01,ACC,52,male,TP,ACC_male_TP
6,ACC-OR-A5J8-TP,ACC-OR-A5J8,TCGA-OR-A5J8-01,ACC,66,male,TP,ACC_male_TP
7,BLCA-4Z-AA80-TP,BLCA-4Z-AA80,TCGA-4Z-AA80-01,BLCA,73,male,TP,BLCA_male_TP
8,BLCA-4Z-AA7N-TP,BLCA-4Z-AA7N,TCGA-4Z-AA7N-01,BLCA,65,male,TP,BLCA_male_TP
9,ACC-OR-A5JC-TP,ACC-OR-A5JC,TCGA-OR-A5JC-01,ACC,37,male,TP,ACC_male_TP


### split the not_training files into validation and test sets

In [16]:
# n_splits = 1, we are not doing cross validation
split = StratifiedShuffleSplit(n_splits=1, test_size=0.5)
for validate_index, test_index in split.split(X_not_train, y_not_train):
    X_validate = X[validate_index]
    X_test = X[test_index]
    
    y_validate = y[validate_index]
    y_test =y[test_index]

In [17]:
validateColDataDF = pd.DataFrame( X_validate, columns=colDataDF.columns)
print(validateColDataDF.shape)
print("\n!!!!!!!!! aedwip not balanced? all from same cohrt is sample to small?")
validateColDataDF

(8, 8)

!!!!!!!!! aedwip not balanced? all from same cohrt is sample to small?


,entity:sample_id,entity:participant_id,tcga_sample_id,Cohort,Age,Gender,sample_type,label
0,ACC-OR-A5JF-TP,ACC-OR-A5JF,TCGA-OR-A5JF-01,ACC,69,female,TP,ACC_female_TP
1,ACC-OR-A5J8-TP,ACC-OR-A5J8,TCGA-OR-A5J8-01,ACC,66,male,TP,ACC_male_TP
2,ACC-OR-A5J9-TP,ACC-OR-A5J9,TCGA-OR-A5J9-01,ACC,22,female,TP,ACC_female_TP
3,ACC-OR-A5J5-TP,ACC-OR-A5J5,TCGA-OR-A5J5-01,ACC,30,male,TP,ACC_male_TP
4,ACC-OR-A5JC-TP,ACC-OR-A5JC,TCGA-OR-A5JC-01,ACC,37,male,TP,ACC_male_TP
5,ACC-OR-A5J3-TP,ACC-OR-A5J3,TCGA-OR-A5J3-01,ACC,23,female,TP,ACC_female_TP
6,ACC-OR-A5J1-TP,ACC-OR-A5J1,TCGA-OR-A5J1-01,ACC,58,male,TP,ACC_male_TP
7,ACC-OR-A5JD-TP,ACC-OR-A5JD,TCGA-OR-A5JD-01,ACC,57,female,TP,ACC_female_TP


In [18]:
y_validate

array([['ACC_female_TP'],
       ['ACC_male_TP'],
       ['ACC_female_TP'],
       ['ACC_male_TP'],
       ['ACC_male_TP'],
       ['ACC_female_TP'],
       ['ACC_male_TP'],
       ['ACC_female_TP']], dtype=object)

In [19]:
testColDataDF = pd.DataFrame( X_test, columns=colDataDF.columns)
print(testColDataDF.shape)
print("\n!!!!!!!!! aedwip not balanced? all from same cohrt is sample to small?")
testColDataDF

(8, 8)

!!!!!!!!! aedwip not balanced? all from same cohrt is sample to small?


,entity:sample_id,entity:participant_id,tcga_sample_id,Cohort,Age,Gender,sample_type,label
0,ACC-OR-A5JI-TP,ACC-OR-A5JI,TCGA-OR-A5JI-01,ACC,22,male,TP,ACC_male_TP
1,ACC-OR-A5J6-TP,ACC-OR-A5J6,TCGA-OR-A5J6-01,ACC,29,female,TP,ACC_female_TP
2,ACC-OR-A5JE-TP,ACC-OR-A5JE,TCGA-OR-A5JE-01,ACC,17,female,TP,ACC_female_TP
3,ACC-OR-A5J7-TP,ACC-OR-A5J7,TCGA-OR-A5J7-01,ACC,30,female,TP,ACC_female_TP
4,ACC-OR-A5J2-TP,ACC-OR-A5J2,TCGA-OR-A5J2-01,ACC,44,female,TP,ACC_female_TP
5,ACC-OR-A5JG-TP,ACC-OR-A5JG,TCGA-OR-A5JG-01,ACC,61,male,TP,ACC_male_TP
6,ACC-OR-A5JA-TP,ACC-OR-A5JA,TCGA-OR-A5JA-01,ACC,53,female,TP,ACC_female_TP
7,ACC-OR-A5JB-TP,ACC-OR-A5JB,TCGA-OR-A5JB-01,ACC,52,male,TP,ACC_male_TP


In [20]:
y_test

array([['ACC_male_TP'],
       ['ACC_female_TP'],
       ['ACC_female_TP'],
       ['ACC_female_TP'],
       ['ACC_female_TP'],
       ['ACC_male_TP'],
       ['ACC_female_TP'],
       ['ACC_male_TP']], dtype=object)

In [21]:
40 * .6 - 2

22.0

# AEDWIP

In [22]:
from sklearn.model_selection import train_test_split

selectRows = colDataDF["label"].isin( labelsAboveThreshold.index )
X = colDataDF.loc[ selectRows  ].to_numpy()
y = colDataDF.loc[ selectRows, ['label'] ].to_numpy()
print("colDataDF.shape:{}".format(colDataDF.shape))
print("X.shape:{}".format(X.shape))
print("y.shape:{}".format(y.shape))
X_train, X_not_train, y_train, y_not_train = train_test_split(X, y,
                                                stratify=y, 
                                                test_size=0.4)

print()
print("X_train.shape:{}".format(X_train.shape))
print("y_train.shape:{}".format(y_train.shape))
print()
print("X_not_train.shape:{}".format(X_not_train.shape))
print("y_not_train.shape:{}".format(y_not_train.shape))

colDataDF.shape:(40, 8)
X.shape:(38, 8)
y.shape:(38, 1)

X_train.shape:(22, 8)
y_train.shape:(22, 1)

X_not_train.shape:(16, 8)
y_not_train.shape:(16, 1)


In [23]:
yTrainDF = pd.DataFrame(y_train, columns=['labels'])
print('yTrainDF')
print(yTrainDF)
yTrainDF.groupby('labels')['labels'].agg('count')

yTrainDF
           labels
0    BLCA_male_TP
1   ACC_female_TP
2    BLCA_male_TP
3   ACC_female_TP
4    BLCA_male_TP
5    BLCA_male_TP
6     ACC_male_TP
7   ACC_female_TP
8    BLCA_male_TP
9    BLCA_male_TP
10   BLCA_male_TP
11  ACC_female_TP
12    ACC_male_TP
13    ACC_male_TP
14   BLCA_male_TP
15  ACC_female_TP
16  ACC_female_TP
17   BLCA_male_TP
18   BLCA_male_TP
19    ACC_male_TP
20    ACC_male_TP
21   BLCA_male_TP


labels
ACC_female_TP     6
ACC_male_TP       5
BLCA_male_TP     11
Name: labels, dtype: int64

In [24]:
yNotTrainDF = pd.DataFrame(y_not_train, columns=['labels'])
print('yNotTrainDF')
print(yNotTrainDF)
yNotTrainDF.groupby('labels')['labels'].agg('count')

yNotTrainDF
           labels
0   ACC_female_TP
1   ACC_female_TP
2   ACC_female_TP
3   ACC_female_TP
4    BLCA_male_TP
5    BLCA_male_TP
6   ACC_female_TP
7    BLCA_male_TP
8    BLCA_male_TP
9     ACC_male_TP
10   BLCA_male_TP
11    ACC_male_TP
12    ACC_male_TP
13    ACC_male_TP
14   BLCA_male_TP
15   BLCA_male_TP


labels
ACC_female_TP    5
ACC_male_TP      4
BLCA_male_TP     7
Name: labels, dtype: int64

In [25]:
X_validate, X_test, y_validate, y_test = train_test_split(X_not_train, y_not_train,
                                                stratify=y_not_train, 
                                                test_size=0.5)

In [26]:
y_validateDF = pd.DataFrame(y_validate, columns=['labels'])
print('y_validateDF')
print(y_validateDF)
y_validateDF.groupby('labels')['labels'].agg('count')

y_validateDF
          labels
0   BLCA_male_TP
1  ACC_female_TP
2   BLCA_male_TP
3    ACC_male_TP
4    ACC_male_TP
5   BLCA_male_TP
6   BLCA_male_TP
7  ACC_female_TP


labels
ACC_female_TP    2
ACC_male_TP      2
BLCA_male_TP     4
Name: labels, dtype: int64

In [27]:
y_testDF = pd.DataFrame(y_test, columns=['labels'])
print('y_test')
print(y_testDF)
y_testDF.groupby('labels')['labels'].agg('count')

y_test
          labels
0   BLCA_male_TP
1  ACC_female_TP
2  ACC_female_TP
3    ACC_male_TP
4    ACC_male_TP
5   BLCA_male_TP
6  ACC_female_TP
7   BLCA_male_TP


labels
ACC_female_TP    3
ACC_male_TP      2
BLCA_male_TP     3
Name: labels, dtype: int64

In [ ]:
aedwip

make sure we have correct sort order for deseq col data

